In [7]:
import win32com.client as win32
import os
import psutil
import subprocess

from dataclasses import dataclass

In [8]:
# Set environment variables
OUTLOOK_APP_PATH = '"C:/Program Files/Microsoft Office/root/Office16/OUTLOOK.EXE"'
OUTLOOK_APP_NAME = 'OUTLOOK.EXE'

# Email dataclass
@dataclass
class Email:
    sendTo: str
    subject: str
    body: str

# Item dataclass
@dataclass
class Item:
    modifiedDate: str  # TODO: Change to date object
    itemNo: str
    itemName: str
    supplierName: str
    
    def createMessage(self) -> str:
        return f"""
        Dear {self.supplierName}
        
        Please send us new data for {self.itemName} [#{self.itemNo}],
        which has expired since {self.modifiedDate}.
        """


In [9]:
def open_outlook(path: str) -> None:
    """ Tries to open Outlook App from given path
    """
    try:
        subprocess.Popen(OUTLOOK_APP_PATH)
    except FileNotFoundError as e:
        print(f"Error: File not found at {OUTLOOK_APP_PATH}")

def app_is_running(app_name: str) -> bool:
    """ Checks running processes to see if a given app is running.
    Returns:
        True -- if app is running else False
    """
    # Get names for all running processes
    running_processes = (psutil.Process(pid).name() for pid in psutil.pids())
        
    for process in running_processes:
        if app_name in process:
            return True

    return False

def create_draft(mail: Email) -> None:
    """Takes an Email dataclass and saves a draft email.
    """
    outlook = win32.Dispatch('outlook.application')
    draft = outlook.CreateItem(0)
    draft.To = mail.sendTo
    draft.Subject = mail.subject
    draft.body = mail.body
    draft.save()


In [10]:
# Make sure we've got an email client running
if not app_is_running(OUTLOOK_APP_NAME):
    print("App not running, opening now.")
    open_outlook(OUTLOOK_APP_PATH)
else:
    print(f"{OUTLOOK_APP_NAME} is already running.")

OUTLOOK.EXE is already running.


In [11]:
# Create a test item
test_item = Item(
    '2019/04/14',
    '1234',
    'Ice Cream',
    'Mr. Freeze',
)

# Create a test email
test_mail = Email(
    'test@example.com',
    'Dataclass Email Test',
    test_item.createMessage(),
)

In [12]:
create_draft(test_mail)